In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import argparse, sys
import numpy as np
from datetime import datetime
import shutil

from parser.get_parser import get_parser
from utils import *
from model.image_model import *
import sys
import time

In [4]:
def accuracy(logit, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    output = F.softmax(logit, dim=1)
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def train(train_loader, epoch, model, optimizer, criterion):
    
    train_total=0
    train_correct=0
    running_loss = 0.0

    #other datasets
    for i, (images, labels, _, _, _, indexes) in enumerate(train_loader):

    # # colthing-1m
    # for i, (images, labels) in enumerate(train_loader):  
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        logits = model(images)
        prec1, _ = accuracy(logits, labels, topk=(1, 5))
        train_total+=1
        train_correct+=prec1

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每 100 个 batch 输出一次
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

def evaluate(test_loader, model):
    model.eval()    # Change model to "eval" mode.
    correct1 = 0
    total1 = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = Variable(images).cuda()
            logits1 = model(images)
            outputs1 = F.softmax(logits1, dim=1)
            _, pred1 = torch.max(outputs1.data, 1)
            total1 += labels.size(0)
            correct1 += (pred1.cpu() == labels).sum()

 
    acc1 = 100*float(correct1)/float(total1)
    return acc1

In [5]:
datasets = ["cifar10", "mnist"]
noise_rates = ["0.2"]
coreset_sizes = ["0", "0.1"]
for noise_rate in noise_rates:
    for coreset_size in coreset_sizes:
        save_path = "./result/" + noise_rate + "_" + coreset_size + "/"
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        for dataset in datasets:
            # sys.argv = ["--dataset", dataset, "--data_root", "/data2/wyh-dataset/image-dataset", "--train_batch_size", "128", "--num_workers", "3", "--noise_rate", noise_rate, "--coreset_size", coreset_size]
            parser = get_parser()
            if dataset == "clothing-1m":
                args = parser.parse_args(args=["--dataset", dataset, "--data_root", "/data2/wyh-dataset/image-dataset", "--train_batch_size", "32", "--test_batch_size", "32", "--num_workers", "4", "--noise_rate", noise_rate, "--coreset_size", coreset_size])
            else:
                args = parser.parse_args(args=["--dataset", dataset, "--data_root", "/data2/wyh-dataset/image-dataset", "--train_batch_size", "128", "--test_batch_size", "128", "--num_workers", "4", "--noise_rate", noise_rate, "--coreset_size", coreset_size])

            train_dataset, val_dataset, test_dataset, num_classes = get_noisy_dataset(args)
            input_channel, forget_rate, args.top_bn, args.epoch_decay_start, args.n_epoch = get_dataset_params(args)
            model = CNN(input_channel=input_channel, n_outputs=num_classes)
            model.cuda()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失
            train_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, drop_last=True, num_workers=args.num_workers, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=args.test_batch_size, drop_last=True, num_workers=args.num_workers, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=args.test_batch_size, drop_last=True, num_workers=args.num_workers, shuffle=False)
            txtfile=save_path + dataset + ".txt"
            start = time.time()
            for epoch in range(0, args.n_epoch):
                model.train()
                train(train_loader, epoch, model, optimizer, criterion)
                torch.cuda.empty_cache()
                test_acc = evaluate(test_loader, model)
                print("Epoch [%d/%d] Test Accuracy on the %s test images: Model %.4f %%" % (epoch+1, args.n_epoch, len(test_dataset), test_acc))

                # with open(txtfile, "a") as myfile:
                    # myfile.write(str(datetime.now()) + ':' + str(int(epoch)) + ": " + str(test_acc) + "\n")
            end = time.time()
            # test_acc = evaluate(test_loader, model)
            print(" %.4f s" % (end-start))
            # with open(txtfile, "a") as myfile:
                # myfile.write("Time: " + str(end-start) + "\n\n")

[Epoch 1, Batch 100] Loss: 2.2219
[Epoch 1, Batch 200] Loss: 2.3463
[Epoch 1, Batch 300] Loss: 2.2903
Epoch [1/50] Test Accuracy on the 10000 test images: Model 25.0801 %
[Epoch 2, Batch 100] Loss: 2.1368
[Epoch 2, Batch 200] Loss: 2.0896
[Epoch 2, Batch 300] Loss: 2.0733
Epoch [2/50] Test Accuracy on the 10000 test images: Model 31.0998 %
[Epoch 3, Batch 100] Loss: 2.0476
[Epoch 3, Batch 200] Loss: 2.0289
[Epoch 3, Batch 300] Loss: 2.0209
Epoch [3/50] Test Accuracy on the 10000 test images: Model 32.1915 %
[Epoch 4, Batch 100] Loss: 2.0344
[Epoch 4, Batch 200] Loss: 2.0320
[Epoch 4, Batch 300] Loss: 2.0494
Epoch [4/50] Test Accuracy on the 10000 test images: Model 30.9095 %
[Epoch 5, Batch 100] Loss: 2.0622
[Epoch 5, Batch 200] Loss: 2.1022
[Epoch 5, Batch 300] Loss: 2.0716
Epoch [5/50] Test Accuracy on the 10000 test images: Model 32.6022 %
[Epoch 6, Batch 100] Loss: 2.0627
[Epoch 6, Batch 200] Loss: 2.1071
[Epoch 6, Batch 300] Loss: 2.0732
Epoch [6/50] Test Accuracy on the 10000 tes

In [6]:
import torch

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   89820 KB |    1135 MB |  230163 GB |  230163 GB |
|       from large pool |   83840 KB |    1129 MB |  229887 GB |  229887 GB |
|       from small pool |    5980 KB |       7 MB |     276 GB |     276 GB |
|---------------------------------------------------------------------------|
| Active memory         |   89820 KB |    1135 MB |  230163 GB |  230163 GB |
|       from large pool |   83840 KB |    1129 MB |  229887 GB |